In [19]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori,association_rules
from mlxtend.preprocessing import TransactionEncoder
import time
import random

In [20]:
df=pd.read_excel("交易資料集(1).xlsx")
print('原始資料筆數:',len(df))


原始資料筆數: 157396


## Data Preprossing


In [21]:
#刪除數量為零或負值的交易
df=df.drop(index=df[(df.QUANTITY<=0)].index.tolist())
#刪除重複資料
df=df.drop_duplicates()
print('資料處理後資料筆數:',len(df))

資料處理後資料筆數: 111961


In [22]:
#整理同一筆交易的商品
print('總共有',len(pd.unique(df['INVOICE_NO'])),'交易紀錄')
print('總共有',len(pd.unique(df['PRODUCT_TYPE'])),'種商品')

總共有 36498 交易紀錄
總共有 10 種商品


In [23]:
df.head(5)

,INVOICE_NO,CUST_ID,ITEM_ID,ITEM_NO,PRODUCT_TYPE,TRX_DATE,QUANTITY
0,CX47348203,3218,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,2016-07-26,2500
1,CX47346522,2470,3326781,AU80610006237AASLBX9,CPU / MPU,2016-07-11,50
2,CX47348534,16135,740487,MMBD2837LT1G,DISCRETE,2016-07-27,3000
4,CX47346184,2356,70072,MMBT3906LT1G,DISCRETE,2016-07-06,12000
6,CX47347899,10228,3420352,TMP103AYFFR,LINEAR IC,2016-07-25,3000


In [24]:
# df['INVOICE_NO']==216072965
df.loc[df['INVOICE_NO'] == 216072965] 

,INVOICE_NO,CUST_ID,ITEM_ID,ITEM_NO,PRODUCT_TYPE,TRX_DATE,QUANTITY
135171,216072965,2717205,14427725,NCP45540IMNTWG-H,OTHERS,2016-07-13,3000
136157,216072965,2717205,1433827,IHLP4040DZERR36M01,PEMCO,2016-07-13,500
140702,216072965,2717205,14563956,W25Q16CLSNIGT,MEMORY_EMBEDED,2016-07-13,2500


In [8]:
unique_values = df['INVOICE_NO'].unique()

print(len(unique_values))
invoice = []
for i in unique_values:
    pick_row = df[df['INVOICE_NO'] == i]
    product_type = np.unique(pick_row['PRODUCT_TYPE'])
    invoice.append(product_type)
    print("INVOICE: ", i)
    print("product_type: ", product_type)


36498
INVOICE:  CX47348203
product_type:  ['MEMORY_EMBEDED']
INVOICE:  CX47346522
product_type:  ['CPU / MPU']
INVOICE:  CX47348534
product_type:  ['DISCRETE']


INVOICE:  CX47346184
product_type:  ['DISCRETE']
INVOICE:  CX47347899
product_type:  ['LINEAR IC' 'LOGIC IC' 'OTHERS']
INVOICE:  CX47346191
product_type:  ['OPTICAL AND SENSOR']
INVOICE:  CX47346636
product_type:  ['DISCRETE']
INVOICE:  CX47348680
product_type:  ['DISCRETE']
INVOICE:  CX47345496
product_type:  ['CHIPSET / ASP']
INVOICE:  CX47348656
product_type:  ['DISCRETE' 'LINEAR IC' 'LOGIC IC' 'OPTICAL AND SENSOR']
INVOICE:  CX47347745
product_type:  ['LINEAR IC' 'LOGIC IC']
INVOICE:  CX47348078
product_type:  ['LINEAR IC']
INVOICE:  CX47346899
product_type:  ['LINEAR IC' 'MEMORY_EMBEDED' 'MEMORY_SYSTEM']
INVOICE:  CX47347172
product_type:  ['CPU / MPU']
INVOICE:  CX47347047
product_type:  ['PEMCO']
INVOICE:  CX47346157
product_type:  ['LINEAR IC']
INVOICE:  CX47345910
product_type:  ['DISCRETE']
INVOICE:  CX47346143
product_type:  ['DISCRETE']
INVOICE:  CX47348630
product_type:  ['DISCRETE']
INVOICE:  CX47345591
product_type:  ['DISCRETE' 'PEMCO']
INVOICE:  CX47346492
product_type

In [9]:
invoice

[array(['MEMORY_EMBEDED'], dtype=object),
 array(['CPU / MPU'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['LINEAR IC', 'LOGIC IC', 'OTHERS'], dtype=object),
 array(['OPTICAL AND SENSOR'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['CHIPSET / ASP'], dtype=object),
 array(['DISCRETE', 'LINEAR IC', 'LOGIC IC', 'OPTICAL AND SENSOR'],
       dtype=object),
 array(['LINEAR IC', 'LOGIC IC'], dtype=object),
 array(['LINEAR IC'], dtype=object),
 array(['LINEAR IC', 'MEMORY_EMBEDED', 'MEMORY_SYSTEM'], dtype=object),
 array(['CPU / MPU'], dtype=object),
 array(['PEMCO'], dtype=object),
 array(['LINEAR IC'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['DISCRETE', 'PEMCO'], dtype=object),
 array(['DISCRETE', 'LINEAR IC'], dtype=object),
 array(['LOGIC IC'], dtype=object),
 array(['CPU / MPU'], dtype=object),
 

In [10]:
te = TransactionEncoder()
te_ary = te.fit(invoice).transform(invoice)
te.columns_
df = pd.DataFrame(te_ary, columns=te.columns_)

In [11]:
df

,CHIPSET / ASP,CPU / MPU,DISCRETE,LINEAR IC,LOGIC IC,MEMORY_EMBEDED,MEMORY_SYSTEM,OPTICAL AND SENSOR,OTHERS,PEMCO
0,False,False,False,False,False,True,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False
3,False,False,True,False,False,False,False,False,False,False
4,False,False,False,True,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...
36493,False,False,False,False,False,False,False,False,False,True
36494,False,False,False,False,False,False,False,False,False,True
36495,False,False,False,False,True,False,False,False,False,False
36496,False,False,True,False,False,False,False,False,False,False


## 關聯規則


In [12]:
sup=np.arange(0.002,0.006,0.001)
conf=np.arange(0.002,0.006,0.001)

ar_support=[]
ar_confidence=[]
ar_time=[]
ar_count=[]
for min_support_sup in sup:
    for min_threshold_conf in conf:
        start = time.time()
        Support_items=apriori(df, min_support=min_support_sup, use_colnames=True)
        apriori_association_rules = association_rules(Support_items, metric = 'confidence', min_threshold=min_threshold_conf)
        end = time.time()
        ar_support.append(min_support_sup)
        ar_confidence.append(min_threshold_conf)
        ar_time.append(end - start)
        ar_count.append(len(apriori_association_rules))
        apriori_association_rules
df_apiori_dataframe=pd.DataFrame({'support':ar_support,'confidence':ar_confidence,'time':ar_time,'count':ar_count})
df_apiori_dataframe


,support,confidence,time,count
0,0.002,0.002,0.037898,146
1,0.002,0.003,0.052861,146
2,0.002,0.004,0.043881,146
3,0.002,0.005,0.166555,146
4,0.003,0.002,0.067818,72
5,0.003,0.003,0.069813,72
6,0.003,0.004,0.027925,72
7,0.003,0.005,0.031914,72
8,0.004,0.002,0.028922,36
9,0.004,0.003,0.028924,36


## 關聯規則輸出存入


In [13]:
apriori_association_rules.to_csv('apriori_association_rules.csv')

In [14]:
import re
#做資料處理
def str(x):
    return re.sub("frozenset|\'|\\{|\\}|\\(|\\)","",x)
df_dataframe=pd.read_csv('apriori_association_rules.csv',index_col=0)
df_dataframe['antecedents']=df_dataframe['antecedents'].apply(lambda x: str(x))
df_dataframe['consequents']=df_dataframe['consequents'].apply(lambda x: str(x))

df_dataframe

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,CHIPSET / ASP,CPU / MPU,0.044112,0.110746,0.005343,0.121118,1.093658,0.000458,1.011802,0.089589
1,CPU / MPU,CHIPSET / ASP,0.110746,0.044112,0.005343,0.048243,1.093658,0.000458,1.004341,0.096302
2,CPU / MPU,LINEAR IC,0.110746,0.260398,0.007672,0.069273,0.266026,-0.021166,0.794650,-0.756254
3,LINEAR IC,CPU / MPU,0.260398,0.110746,0.007672,0.029461,0.266026,-0.021166,0.916248,-0.788602
4,CPU / MPU,LOGIC IC,0.110746,0.121541,0.005206,0.047006,0.386754,-0.008254,0.921789,-0.640688
5,LOGIC IC,CPU / MPU,0.121541,0.110746,0.005206,0.042831,0.386754,-0.008254,0.929047,-0.643494
6,OTHERS,CPU / MPU,0.113403,0.110746,0.005288,0.046630,0.421051,-0.007271,0.932748,-0.607979
7,CPU / MPU,OTHERS,0.110746,0.113403,0.005288,0.047749,0.421051,-0.007271,0.931053,-0.607266
8,LINEAR IC,DISCRETE,0.260398,0.257905,0.034906,0.134049,0.519761,-0.032252,0.856971,-0.555411
9,DISCRETE,LINEAR IC,0.257905,0.260398,0.034906,0.135345,0.519761,-0.032252,0.855372,-0.554579


In [15]:
df_dataframe['antecedents']

0           CHIPSET / ASP
1               CPU / MPU
2               CPU / MPU
3               LINEAR IC
4               CPU / MPU
5                LOGIC IC
6                  OTHERS
7               CPU / MPU
8               LINEAR IC
9                DISCRETE
10               LOGIC IC
11               DISCRETE
12         MEMORY_EMBEDED
13               DISCRETE
14     OPTICAL AND SENSOR
15               DISCRETE
16                 OTHERS
17               DISCRETE
18               LOGIC IC
19              LINEAR IC
20         MEMORY_EMBEDED
21              LINEAR IC
22                 OTHERS
23              LINEAR IC
24    LOGIC IC, LINEAR IC
25     LOGIC IC, DISCRETE
26    LINEAR IC, DISCRETE
27               LOGIC IC
28              LINEAR IC
29               DISCRETE
Name: antecedents, dtype: object

In [16]:
df_dataframe['consequents']

0               CPU / MPU
1           CHIPSET / ASP
2               LINEAR IC
3               CPU / MPU
4                LOGIC IC
5               CPU / MPU
6               CPU / MPU
7                  OTHERS
8                DISCRETE
9               LINEAR IC
10               DISCRETE
11               LOGIC IC
12               DISCRETE
13         MEMORY_EMBEDED
14               DISCRETE
15     OPTICAL AND SENSOR
16               DISCRETE
17                 OTHERS
18              LINEAR IC
19               LOGIC IC
20              LINEAR IC
21         MEMORY_EMBEDED
22              LINEAR IC
23                 OTHERS
24               DISCRETE
25              LINEAR IC
26               LOGIC IC
27    LINEAR IC, DISCRETE
28     LOGIC IC, DISCRETE
29    LOGIC IC, LINEAR IC
Name: consequents, dtype: object

## 推薦商品


In [17]:
# recommended_products=input('請輸入商品名稱:')
# print(recommended_products)